In [ ]:
import plotly.express as px
import numpy as np
import duckdb
from tqdm import tqdm
import plotly

In [ ]:
import sys
sys.path.append("/home/ubuntu/sky_workdir/encoding-schemes")

from encoding_schemes import get_deterministic_adherence_fn

In [ ]:
import ray

ray.init()

In [ ]:
import os
import psycopg2
import json

conn_string = os.environ["SUPABASE_CONNECTION_URL"]

conn = psycopg2.connect(conn_string)

import pandas as pd

In [ ]:
sel_str = """
-- NuminaMath CoT Rerun
 (
     (data->'experiment_tags'->'numina_math_cot_rerun')::BOOL
     AND (NOT (data->'force_overwrite')::BOOL OR data->'force_overwrite' IS NULL)
     AND (
         (data->'experiment_params'->'sampling_params'->'n')::INT = 4
         OR ((data->'experiment_params'->'model')::TEXT LIKE '%gpt%' AND (data->'experiment_params'->'sft_params'->'batch_size')::INT != 48)
     )
  )
"""

df = pd.read_sql(f"""
SELECT * FROM public.encoding_schemes 
    WHERE 

{sel_str}


ORDER BY created_at DESC
""", conn)

df.head()

In [ ]:
root_dir = "/home/ubuntu/sky_workdir/encoding-schemes/output"

In [ ]:
l_examples = df.to_dict('records')

# Code

In [ ]:

def bootstrap_ci(data, statistic=np.mean, alpha=0.05, n_boot=10_000, random_state=None):
    """
    Returns (point_estimate, low_CI, high_CI) for given 1D data.
    Works with bool, int, or float data.
    """
    x = np.asarray(data).astype(float)  # ensure numeric
    x = x[~np.isnan(x)]
    if len(x) == 0:
        raise ValueError("No valid data for bootstrapping.")

    rng = np.random.default_rng(random_state)
    n = len(x)

    # Draw bootstrap samples
    idx = rng.integers(0, n, size=(n_boot, n))
    samples = x[idx]

    # Apply statistic row-wise
    stats = np.apply_along_axis(statistic, 1, samples)

    point = statistic(x)
    lo = np.percentile(stats, 100 * (alpha / 2))
    hi = np.percentile(stats, 100 * (1 - alpha / 2))
    return point, lo, hi

In [ ]:
import tiktoken

encoding = tiktoken.get_encoding("cl100k_base")


@ray.remote
def count_tokens_from_messages(s):
    try:
        return len(encoding.encode(s, disallowed_special=()))
    except ValueError as e:
        print(e)
        return 0


In [ ]:
@ray.remote(num_cpus=2, memory=32 * 1024 * 1024 * 1024)
def compute_translation_token_count(example, df_data):
    sys.path.append("/home/ubuntu/sky_workdir/encoding-schemes")

    from orchestration.experiment_meta_saver import compute_experiment_hash
    from utils.io_utils import read_large_parquet

    from transformers import AutoTokenizer

    model = example["data"]["experiment_params"]["model"]
    if "gpt" in model or "claude" in model:
        print(f"Overriding tokenizer for {model} with gpt-oss 120b tokenizer because it was detected as a GPT/Claude model!")
        model = "openai/gpt-oss-120b"

    tokenizer = AutoTokenizer.from_pretrained(model)

    return df_data['reference_solution'].map(lambda x: len(tokenizer.encode(x)))


In [ ]:
from tqdm import tqdm

In [ ]:
def compute_ci_cols(example, df, col_name, transformation_fn):
    s_transformed = transformation_fn(df[col_name])
    if np.isscalar(s_transformed) and np.isnan(s_transformed):
        print(f"Warning: {col_name} was all NaN, ignoring!")
        return
    mid, lo, hi = bootstrap_ci(s_transformed)
    example[col_name] = mid
    example[f'{col_name}_low_ci'] = mid - lo
    example[f'{col_name}_hi_ci'] = hi - mid

In [ ]:
def compute_multi_row_transformation(df, row_transform_fn, col_name, agg_fn):
    df[col_name] = df.apply(row_transform_fn, axis=1)

In [ ]:

def _score_rollouts(rollouts):
    # rollouts: list/array of rollout sequences; may also be None/np.nan/scalar
    if rollouts is None or (isinstance(rollouts, float) and np.isnan(rollouts)):
        return np.nan

    vals = []
    for r in rollouts:
        # skip None/NaN
        if r is None or (isinstance(r, float) and np.isnan(r)):
            continue
        vals.append(-np.nansum(r))
    return np.nanmean(vals) if len(vals) else np.nan


def patch_gpt_api_log_loss(example):
    experiment_hash = example['experiment_hash']

    translation_loss = os.path.join(root_dir, experiment_hash, "data", f"validation_reverse_translation_math500_meta.json")
    with open(translation_loss, "r") as fp:
        example["backtranslation_gt_logprobs"] = translation_loss["valid_loss"]

    # need to be validation loss on 512k...
    validation_loss = os.path.join(root_dir, experiment_hash, "data", f"validation_reverse_translation_math500_meta.json")
    with open(validation_loss, "r") as fp:
        example["backtranslation_gt_logprobs"] = translation_loss["valid_loss"]


@ray.remote
def process_single_example(example):
    experiment_hash = example['experiment_hash']
    
    target_path = os.path.join(root_dir, example['experiment_hash'], "data", "joined_output.parquet")
    if not os.path.exists(target_path):
        print(f"!!!!!! {target_path} missing !!!!!!!")
        return example
    
    df_data = pd.read_parquet(target_path)

    d_col_to_transform = {
        'cot_gt_logprobs' : lambda s: np.nansum(s.map(_score_rollouts)),
        'generated_cot_is_correct' : np.mean,  # was np.mean
        'backtranslation_gt_logprobs' : lambda s: np.nanmean(s.map(_score_rollouts)),
        'backtranslation_bleu_scores' : np.mean,  # was np.mean
        'generated_cot_adhered_encoding_style': np.mean  # was np.mean
    }
    for col, fn in d_col_to_transform.items():
        if col not in df_data:
            print(col)
            print(df_data.head())
            print(example)
            raise Exception(str(col) + "\n" + str(df_data.head()) + "\n" + str(example))

        compute_ci_cols(example, df_data, col, fn)

    df_data["num_tokens_translation_output"] = ray.get(compute_translation_token_count.remote(example, df_data))

    d_and_cols = {
        'adherent_and_correct': (
            lambda r: np.nanmean( \
                np.array(r['generated_cot_is_correct']).astype(bool) & \
                np.array(r['generated_cot_adhered_encoding_style']).astype(bool) \
            ),
            np.nanmean
        ),
        'total_translation_loss': (
            lambda r: np.nanmean( \
                np.array(r['num_tokens_translation_output']) * \
                np.array(np.nanmean(_score_rollouts(r['backtranslation_gt_logprobs'])), dtype=np.float64) \
            ),
            np.nanmean
        ),
    }
    for col, (transform_fn, agg_fn) in d_and_cols.items():
        compute_multi_row_transformation(df_data, transform_fn, col, agg_fn)
        if df_data[col].isna().sum() != len(df_data):
            compute_ci_cols(example, df_data, col, lambda x: x)

    if "gpt" in example["data"]["experiment_params"]["model"] and "use_api_sft_model_for_sampling" in example["data"]["experiment_params"]:
        with open(os.path.join(root_dir, example['experiment_hash'], "data", f"validation_reverse_translation_math500_meta.json"), "r") as fp:
            d_model_meta = json.load(fp)

        example["backtranslation_gt_logprobs"] = d_model_meta["valid_loss"]
        example["total_translation_loss"] = d_model_meta["valid_loss"] * np.nanmean(df_data["num_tokens_translation_output"])
        example["total_translation_loss_low_ci"] = 0.0
        example["total_translation_loss_hi_ci"] = 0.0

    pretraining_prevalence_path = os.path.join('/home/ubuntu/sky_workdir/encoding-schemes', 'output', experiment_hash, 'data', 'num_pretraining_4grams_redpajama.json')
    if os.path.exists(pretraining_prevalence_path):
        with open(pretraining_prevalence_path, "r") as fp:
            d_pretraining_prevalence = json.load(fp)

        example["pretraining_prevalence"] = d_pretraining_prevalence["num_occurrences"]

    for col in df_data.columns:
        example[f"{col}_df"] = df_data[col]

    return example


l_new_examples = [None for _ in range(len(l_examples))]

for i, example in tqdm(enumerate(l_examples)):
    # l_examples[i] = process_single_example(example)
    l_new_examples[i] = process_single_example.remote(example)

for i, example in tqdm(enumerate(l_new_examples)):
    try:
        l_new_examples[i] = ray.get(example)
    except ray.exceptions.RayTaskError as e:
        l_new_examples[i] = l_examples[i]
        print(e)

l_examples = l_new_examples

In [ ]:
def humanize_number(num: float) -> str:
    """
    Converts a number into a human-readable string with k, M, or B suffixes.
    
    Args:
        num (float): The number to format.
    
    Returns:
        str: Human-readable string representation.
    """
    if num >= 1_000_000_000:
        return f"{num / 1_000_000_000:.1f}B"
    elif num >= 1_000_000:
        return f"{num / 1_000_000:.1f}M"
    elif num >= 1_000:
        return f"{num / 1_000:.1f}k"
    else:
        return str(num)

In [ ]:
import re

def parse_params(model):
    if 'gpt' in model:
        if 'nano' in model:
            return 0
        elif 'mini' in model:
            return 1
        else:
            return 2


    if 'claude' in model:
        if 'haiku' in model:
            return 3
        elif '3-opus' in model:
            return 4
        elif '3-5' in model:
            return 5
        elif 'sonnet-4' in model:
            return 6
        else:
            return 7
    
    return int(re.search("([0-9]+)B", model).group(1))

In [ ]:
df_viz = pd.DataFrame(l_examples)

orig_len = len(df_viz)

# df_viz = df_viz[df_viz['cot_gt_logprobs'].notna()]

new_len = len(df_viz)
if orig_len != new_len:
    print(f"Lost {orig_len - new_len} examples from na logprobs")

df_viz['encoding_scheme'] = df_viz['data'].map(lambda x: x['experiment_params']['encoding_scheme'])
df_viz['model'] = df_viz['data'].map(lambda x: x['experiment_params']['model'])

try:
    df_viz['model_size'] = df_viz['model'].map(parse_params)
except Exception as e:
    print(e)
df_viz['input_type'] = df_viz['data'].map(lambda x: "_".join(x['experiment_name'].split("_")[:2]))

df_viz['n_few_shot_examples'] = df_viz['data'].map(lambda x: x['experiment_params'].get('n_few_shot_examples', None))

df_viz['Adherence Calculation Method'] = df_viz['encoding_scheme'].map(lambda x: get_deterministic_adherence_fn(x, None) is not None).map({ True: 'deterministic', False: 'Sonnet 4 judge'})

try:
    df_viz['total_train_tok'] = df_viz['n_total_train_tok'].map(humanize_number)
except Exception as e:
    print(e)


In [ ]:
filter_set = ['math_cot']

In [ ]:
df_viz_tmp = df_viz[df_viz['input_type'].isin(filter_set)]
df_viz_tmp = df_viz_tmp[df_viz_tmp['model'] != 'Qwen/Qwen2.5-7B']

df_viz_tmp = df_viz_tmp.sort_values([
    'model_size',
    'adherent_and_correct'
])

df_viz_tmp = df_viz_tmp.astype({'n_few_shot_examples': str})

df_viz_tmp['encoding_scheme'] = df_viz_tmp['encoding_scheme'].map(lambda s: s.split("speaking_")[-1])

# Paper plots

In [ ]:
d_mapping = {
    "baseline": ["zero_shot", "identity"],
    "letter mutation": [
        "reverse_letters_in_each_word",
        "swap_even_odd_letters_in_each_word",
        "reverse_fibonacci_indices_in_each_word",
        "letter_to_word_with_dot",
        "dot_between_chars",
        "space_between_chars",
    ],
    "language deletion": ["remove_all_verbs", "remove_all_nouns"],
    "language translation": [
        "French","Chinese","Korean","Russian","Arabic","Adyghe",
        "Morse_code","Python","enterprise_Java",
    ],
    "algorithmic cipher": [
        "rot13_cipher","base64_cipher","base64_2x_cipher",
        "base64_3x_cipher","caesar_cipher","gzip_to_base64_encoded",
    ],
    "themed reasoning": [
        "paraphrase_naive",
        "pirate_speak",
        "leet_speak",
        "yoda_speak",
        "shakespearean_text",
    ],
    "extraneous content": [
        "insert_tweet",
        "python_snippet_comment",
        "croissant_news_article",
        "math_textbook_article",
        "five_emojis",
    ],
    "delete inf.": [
        "replace_math_content_with_black_box"
    ]
}

l_themed_encodings = [
    "remove_all_verbs",
    "paraphrase_naive",
    "pirate_speak",
    "leet_speak",
    "yoda_speak",
    "shakespearean_text",
    "insert_tweet",
    "python_snippet_comment",
    "croissant_news_article",
    "math_textbook_article",
    "five_emojis",
    "replace_math_content_with_black_box",
    "reverse_letters_in_each_word_no_math_expressions",
    "reverse_letters_in_each_word_only_math_expressions"
]

l_ignore_languages = [
    "Russian",
    "Chinese",
    # "Korean"
]

In [ ]:
import plotly.express as px
import plotly.colors as pc

def sample_colorscale(colorscale_name: str, n: int):
    """
    Sample N equally spaced hex colors from a Plotly continuous colorscale.

    Parameters
    ----------
    colorscale_name : str
        Name of a Plotly built-in continuous colorscale (e.g., "Viridis", "Blues").
    n : int
        Number of samples to return.

    Returns
    -------
    list of str
        List of hex color strings.
    """
    if n < 1:
        raise ValueError("n must be >= 1")
    colorscale = px.colors.get_colorscale(colorscale_name)
    # equally spaced points in [0,1]
    vals = [i/(n-1) if n > 1 else 0.5 for i in range(n)]
    return [pc.sample_colorscale(colorscale, v)[0] for v in vals]

l_gpt_gradient = sample_colorscale("Emrld", 4 + 1)
d_gpt_gradient = {
    model : color
    for model, color in zip(
        ['gpt-4.1-nano-2025-04-14', 'gpt-4.1-mini-2025-04-14', 'gpt-4.1-2025-04-14', 'gpt-5-chat-latest'],
        l_gpt_gradient[:-1]
    )
}

l_claude_gradient = sample_colorscale("Purpor", 3 + 1)
d_claude_gradient = {
    model : color
    for model, color in zip(
        ['claude-3-opus-20240229', 'claude-3-5-sonnet-20241022', 'claude-sonnet-4-20250514'],
        l_claude_gradient[1:]
    )
}

l_qwen_gradient = sample_colorscale("Oryel", 3 + 1)
d_qwen_gradient = {
    model : color
    for model, color in zip(
        ['Qwen2.5-3B-Instruct', 'Qwen2.5-7B-Instruct', 'Qwen2.5-14B-Instruct'],
        l_qwen_gradient[1:]
    )
}

In [ ]:
l_star_encodings = [
    "reverse_letters_in_each_word",
    "rot13_cipher",
    "caesar_cipher", # TODO check
 'reverse_fibonacci_indices_in_each_word', # TODO check
 'swap_even_odd_letters_in_each_word', # TODO check
    "Morse_code"
]

In [ ]:
l_fixed_encoding_ordering = [
    # GPT
 'identity',
'dot between chars',
 'rot13 cipher',
 'Korean',
 'reverse letters in each word',
 'letter to word with dot',
     'base64 cipher',
 # 'remove all verbs',

    # placeholder for an empty space
    '',

    # Qwen/Claude
'space between chars',
 'Arabic',
 'caesar cipher',
 'French',
 'Adyghe',
 'Morse code',
 'Python',
 'reverse fibonacci indices in each word',
 'swap even odd letters in each word',
 'base64 3x cipher',
 'base64 2x cipher',
 'remove all nouns',
 'enterprise Java',
 'gzip to base64 encoded'
 
]

l_fixed_encoding_ordering = [s.replace(' ', '_') for s in l_fixed_encoding_ordering]

for i in range(len(l_fixed_encoding_ordering)):
    if l_fixed_encoding_ordering[i] in l_star_encodings:
        l_fixed_encoding_ordering[i] += "*"

In [ ]:
import plotly.graph_objects as go
import pandas as pd

def make_encoding_scheme_bar_plot(
    df,
    y_col="generated_cot_is_correct",
    title="MATH 500 Accuracy",
    y_axis_title="Accuracy",
    x_col="encoding_scheme",
    model_col="model",
    d_mapping=None,
    yaxis_dtick=0.1,
    show_text=True,
    model_order=None,
    font_family="Inter, Arial",
    sort_by_col=None,
    sort_agg="max",
    sort_desc=True
):
    # --- 1) DEFAULT SECTION MAP ---
    if d_mapping is None:
        d_mapping = {
            "Baselines": [
                "identity",
                "zero_shot",
            ],
            "Stylistic text": [
                "paraphrase_naive",
                "pirate_speak",
                "yoda_speak",
                "shakespearean_text",
            ],
            "Distractors": [
                "insert_tweet",
                "python_snippet_comment",
            ],
            "Language": [
                "leet_speak",
                "rot13_cipher",
                "Morse_code"
            ],
            "Inf. content": [
                "replace_math_content_with_black_box"
            ]
        }

    l_color_categories = ["Language", "Inf. content"]

    df_plot = df.copy()

    # --- 2) ORDERING BY d_mapping ---
    full_category_order, section_spans = [], []
    present_set = set(df_plot[x_col].unique())
    cursor = 0
    
    # Create a mapping of encoding schemes to their sections
    scheme_to_section = {}
    for section_name, schemes in d_mapping.items():
        for scheme in schemes:
            scheme_to_section[scheme] = section_name

    # Use the order defined in d_mapping directly
    for section_name, schemes in d_mapping.items():
        # Keep only schemes that are present in the data, but maintain their order
        present = [s for s in schemes if s in present_set]
        if not present:
            continue

        start = cursor
        full_category_order.extend(present)
        cursor = len(full_category_order)
        end = cursor - 1
        section_spans.append((start, end, section_name))

    if not full_category_order:
        full_category_order = list(df_plot[x_col].unique())
        section_spans = [(0, len(full_category_order) - 1, "All")]

    df_plot[x_col] = pd.Categorical(df_plot[x_col], categories=full_category_order, ordered=True)

    # --- PRESERVE MODEL ORDER ---
    if model_order is None:
        model_order = list(dict.fromkeys(df_plot[model_col]))
    df_plot[model_col] = pd.Categorical(df_plot[model_col], categories=model_order, ordered=True)

    # --- ERROR BARS ---
    err_hi_col = f"{y_col}_hi_ci"
    err_lo_col = f"{y_col}_low_ci"
    error_y = err_hi_col if err_hi_col in df_plot.columns else None
    error_y_minus = err_lo_col if err_lo_col in df_plot.columns else None

    def prettify(s: str) -> str:
        s = s.replace("_", "<br>")
        return s

    # --- DETECT WHETHER Y-VALUES ARE PERCENTAGES ---
    y_min, y_max = df_plot[y_col].min(), df_plot[y_col].max()
    is_percent = y_max <= 1.0 and y_min >= 0.0

    if yaxis_dtick is None:
        yaxis_dtick = 0.1 if is_percent else None
    
    # --- CREATE COLOR MAPPINGS ---
    # Assuming you have these dictionaries defined elsewhere
    # If not, you'll need to define them
    original_colors = {**d_qwen_gradient, **d_gpt_gradient}
    
    # Convert colors to greyscale for non-colored categories
    import plotly.colors as pc
    
    def to_greyscale(color_str):
        """Convert a color to greyscale, handles both hex and rgb(r,g,b) formats"""
        import re
        
        # Check if it's an rgb string
        rgb_match = re.match(r'rgb\((\d+),\s*(\d+),\s*(\d+)\)', color_str)
        if rgb_match:
            r, g, b = map(int, rgb_match.groups())
        # Check if it's hex format
        elif color_str.startswith('#'):
            import plotly.colors as pc
            rgb = pc.hex_to_rgb(color_str)
            r, g, b = rgb
        else:
            raise Exception()
        
        # Use luminance formula to convert to greyscale
        grey_val = int(0.299 * r + 0.587 * g + 0.114 * b)
        return f'rgb({grey_val}, {grey_val}, {grey_val})'

    
    # Create the figure using graph_objects for more control
    fig = go.Figure()
    
    # Add traces for each model
    for model in model_order:
        model_data = df_plot[df_plot[model_col] == model].sort_values(x_col)
        
        # Prepare colors for each bar
        colors = []
        for scheme in model_data[x_col]:
            section = scheme_to_section.get(scheme, "")
            colors.append(original_colors[model])
            # if section in l_color_categories:
            #     # Use original color
            #     colors.append(original_colors[model])
            # else:
            #     # Use greyscale version
            #     original_color = original_colors[model]
            #     colors.append(to_greyscale(original_color))
        
        # Add error bars if available
        error_y_dict = None
        if error_y and error_y_minus:
            error_y_dict = dict(
                type='data',
                symmetric=False,
                array=model_data[err_hi_col] if err_hi_col in model_data.columns else None,
                arrayminus=model_data[err_lo_col] if err_lo_col in model_data.columns else None
            )

        l_text = [f"-{(1 - d) * 100 :.0f}%" for d in model_data[y_col]]
        l_text = [f"<b><span style='color:blue'>{t}</span></b>" if 1 - d > 0.2 and encoding != 'zero_shot' else "" for t, d, encoding in zip(l_text, model_data[y_col], model_data['encoding_scheme'])]
        fig.add_trace(go.Bar(
            name=model,
            x=model_data[x_col],
            y=model_data[y_col],
            marker_color=colors,
            # error_y=error_y_dict,
            text=l_text if show_text else None,
            # texttemplate="%{y:.0%}" if (is_percent and show_text) else "%{y:}" if show_text else None,
            textposition="outside" if show_text else "none",
        ))
    
    # Update layout
    fig.update_layout(
        title=title,
        template="plotly_white",
        height=500,
        width=1500,
        barmode="group",
        bargap=0.15,
        bargroupgap=0.05,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.08,
            xanchor="left",
            x=0.01,
            font=dict(size=18)
        ),
        margin=dict(t=0, r=0, b=0, l=0),
        font=dict(family=font_family, size=15),
        title_font=dict(size=22)
    )
    
    # Y axis
    if is_percent:
        fig.update_yaxes(
            title=y_axis_title,
            tickformat=".0%",
            tickmode="linear",
            dtick=yaxis_dtick,
            rangemode="tozero"
        )
    else:
        fig.update_yaxes(
            title=y_axis_title,
            tickmode="linear",
            rangemode="tozero",
            dtick=yaxis_dtick
        )
    
    # X axis
    fig.update_xaxes(
        title="Encoding scheme",
        ticktext=[f"<b>{prettify(lbl)}</b>" for lbl in full_category_order],
        tickvals=full_category_order,
        tickangle=0,
        tickfont=dict(size=15)
    )
    
    # --- SECTION BACKGROUNDS + LABELS ---
    N = len(full_category_order)
    
    def to_xdomain(idx):
        return idx / N
    
    shapes = []
    annotations = []
    for i, (start_idx, end_idx, name) in enumerate(section_spans):
        x0 = to_xdomain(start_idx)
        x1 = to_xdomain(end_idx + 1)
        band = dict(
            type="rect",
            xref="x domain", yref="paper",
            x0=x0, x1=x1, y0=0, y1=1,
            layer="between",
            line=dict(width=0),
            fillcolor="#C4D8E2" if i >= 3 else "rgba(0,0,0,0.03)" if i % 2 == 0 else "rgba(0,0,0,0.00)"
        )
        shapes.append(band)
        annotations.append(dict(
            x=(x0 + x1) / 2, xref="x domain",
            y=1.1, yref="paper",
            text=name, showarrow=False,
            font=dict(size=18, color="black"),
            xanchor="center"
        ))
    
    fig.update_layout(shapes=shapes)
    for a in annotations:
        fig.add_annotation(**a)
    
    return fig


In [ ]:
# --- Compute PGR (percent of identity performance) ---
df_pgr = df_viz_tmp.copy()

# df_pgr = df_pgr[~( (df_pgr['input_type'] == 'mathcot_fewshot') & (df_pgr['encoding_scheme'] == 'identity') )]


df_pgr['model'] = df_pgr['model'].str.split('Qwen/').str[-1]

# df_pgr = df_pgr[~df_pgr['model'].str.contains('gpt')]
# df_pgr = df_pgr[df_pgr['model'].str.contains('14B')]

# Identity baseline per model
identity_baseline = (
    df_pgr[df_pgr["encoding_scheme"] == "identity"]
    .set_index("model")["adherent_and_correct"]
)

df_pgr["identity_value"] = np.maximum(df_pgr["model"].map(identity_baseline), 0.0001)

# Avoid divide-by-zero
# df_pgr = df_pgr[df_pgr["identity_value"] > 0].copy()

# Mean PGR as percentage
df_pgr["PGR_pct"] = (df_pgr["adherent_and_correct"] / df_pgr["identity_value"])

# CI deltas -> percentage deltas relative to identity baseline
# low is negative, high is positive
df_pgr["PGR_pct_hi_ci"]  = (df_pgr["adherent_and_correct_hi_ci"] / df_pgr["identity_value"])
df_pgr["PGR_pct_low_ci"] = (df_pgr["adherent_and_correct_low_ci"]        / df_pgr["identity_value"])

df_pgr.loc[df_pgr["encoding_scheme"] == "identity", ["PGR_pct_hi_ci", "PGR_pct_low_ci"]] = 0.0

fig = make_encoding_scheme_bar_plot(
    df=df_pgr[df_pgr['model'].str.contains('Qwen')],
    # df=df_pgr[df_pgr['model'] == 'gpt-4.1-2025-04-14'],
    y_col="PGR_pct",
    # title="Relative % of responses adherent & correct vs. identity encoding",
    title=None,
    y_axis_title="% of identity adherent & correct",
    yaxis_dtick=0.1,
    sort_by_col="adherent_and_correct"
)

fig.update_yaxes(range=[0.0, 1.0])
fig.update_xaxes(title="Cipher")
# fig.update_yaxes(range=[0.0, 0.35], dtick=0.05)
# fig.update_layout(height=475)
# fig.update_traces(width=0.5)
fig.update_layout(font=dict(size=24))
fig.update_layout(
    xaxis=dict(
        tickfont=dict(
            size=18,         # Specify the font size
        )
    ),
    yaxis=dict(title_font=dict(size=20), tickfont=dict(size=18))
)

fig.add_hline(y=1)

# fig.update_traces(textfont_size=15)

plotly.io.write_image(fig, 'sft_ablations.pdf', format='pdf')

fig.show('png')


In [ ]:
import pandas as pd
import numpy as np

def df_to_latex_table(df_pgr,
                      caption="Model results",
                      label="tab:model-results",
                      percent_decimals=1,
                      float_decimals=2):
    from io import StringIO

    # 1) Build df_export
    df_export = df_pgr.copy()

    df_export['encoding_scheme'] = df_export['encoding_scheme'].str.replace('_', ' ')
    df_export['encoding_scheme'] = df_export['encoding_scheme'].map(
        lambda x: 'direct answering' if x == 'zero shot' else x
    )

    df_export = df_export[
        ['model', 'encoding_scheme', 'PGR_pct', 'generated_cot_is_correct',
         'generated_cot_adhered_encoding_style', 'backtranslation_bleu_scores',
         'Adherence Calculation Method']
    ].rename(columns={
        'encoding_scheme': 'encoding scheme',
        'PGR_pct': '\\% of identity adherent + coherent',
        'generated_cot_is_correct': 'MATH500 accuracy',
        'generated_cot_adhered_encoding_style': 'cipher adherence',
        'backtranslation_bleu_scores': 'translation BLEU',
    })

    # 2) Format numbers
    df_fmt = df_export.copy()

    col_pct = '\\% of identity adherent + coherent'
    if pd.api.types.is_numeric_dtype(df_fmt[col_pct]):
        s = df_fmt[col_pct].astype(float)
        if np.nanmax(s.values) <= 1.5:  # treat as fraction
            s = s * 100.0
        df_fmt[col_pct] = s.map(lambda x: (f"{x:.{percent_decimals}f}\\%" if pd.notna(x) else ""))

    numeric_cols = [
        'MATH500 accuracy',
        'cipher adherence',
        'translation BLEU',
    ]
    for c in numeric_cols:
        if c in df_fmt.columns and pd.api.types.is_numeric_dtype(df_fmt[c]):
            df_fmt[c] = df_fmt[c].map(lambda x: (f"{x:.{float_decimals}f}" if pd.notna(x) else ""))

    # 3) Build LaTeX manually with tabularx
    buffer = StringIO()
    col_names = df_fmt.columns.tolist()

    # Choose alignments: first two + last = text (wrap), others numeric
    col_spec = " | ".join(["p{" + f"{0.75 / len(col_names):.2f}" + "\\linewidth}" for _ in range(len(col_names))])

    # Write header
    buffer.write("\\begin{longtable}[]")
    buffer.write("{" + col_spec + "}\n")

    # Column headers
    buffer.write(" & ".join(col_names) + " \\\\\n")
    buffer.write("\\hline\n")

    # Rows
    for _, row in df_fmt.iterrows():
        buffer.write(" & ".join(str(x) for x in row.tolist()) + " \\\\\n")

    buffer.write("\\caption{" + caption + "}\n")
    buffer.write("\\label{" + label + "}\n")
    buffer.write("\\end{longtable}\n")

    latex = buffer.getvalue()
    print(latex)
    return latex



df_to_latex_table(df_pgr[df_pgr['PGR_pct'].notna() & df_pgr['model'].str.contains('Qwen') & df_pgr['encoding_scheme'].isin(
[
                    "paraphrase_naive",
                "pirate_speak",
                "yoda_speak",
                "shakespearean_text",
                "insert_tweet",
                "python_snippet_comment",
                "leet_speak",
                "replace_math_content_with_black_box"
    ]
)
    ], caption="TODO CAPTION", label="detailed_sft_results_table")

None